In [1]:
# 引入rich打印
import sys
sys.path.append('../..')

from rich import print
from llm_common.set_env import set_open_ai_key,print_current_path,print_input,gothrough

set_open_ai_key()
print_current_path()


PlZPbSg-Qu9Kr45iP0pV_0rhlAy2QA

In [22]:
from langchain_community.chat_message_histories import ChatMessageHistory

demo_ephemeral_chat_history = ChatMessageHistory()

demo_ephemeral_chat_history.add_user_message("Hey there! I'm Nemo.")
demo_ephemeral_chat_history.add_ai_message("Hello!")
demo_ephemeral_chat_history.add_user_message("How are you today?")
demo_ephemeral_chat_history.add_ai_message("Fine thanks!")

print(demo_ephemeral_chat_history.messages)


[
    HumanMessage(content="Hey there! I'm Nemo.", additional_kwargs={}, response_metadata={}),
    AIMessage(content='Hello!', additional_kwargs={}, response_metadata={}),
    HumanMessage(content='How are you today?', additional_kwargs={}, response_metadata={}),
    AIMessage(content='Fine thanks!', additional_kwargs={}, response_metadata={})
]

In [21]:
from langchain_community.chat_message_histories import FileChatMessageHistory

demo_ephemeral_chat_history_file = FileChatMessageHistory(file_path='./chat_history.txt')

demo_ephemeral_chat_history_file.add_user_message("Hey there! I'm Nemo.")
demo_ephemeral_chat_history_file.add_ai_message("Hello!")
demo_ephemeral_chat_history_file.add_user_message("How are you today?")
demo_ephemeral_chat_history_file.add_ai_message("Fine thanks!")

print(demo_ephemeral_chat_history_file.messages)

[
    HumanMessage(content="Hey there! I'm Nemo.", additional_kwargs={}, response_metadata={}),
    AIMessage(content='Hello!', additional_kwargs={}, response_metadata={}),
    HumanMessage(content='How are you today?', additional_kwargs={}, response_metadata={}),
    AIMessage(content='Fine thanks!', additional_kwargs={}, response_metadata={}),
    HumanMessage(content="Hey there! I'm Nemo.", additional_kwargs={}, response_metadata={}),
    AIMessage(content='Hello!', additional_kwargs={}, response_metadata={}),
    HumanMessage(content='How are you today?', additional_kwargs={}, response_metadata={}),
    AIMessage(content='Fine thanks!', additional_kwargs={}, response_metadata={}),
    HumanMessage(content="Hey there! I'm Nemo.", additional_kwargs={}, response_metadata={}),
    AIMessage(content='Hello!', additional_kwargs={}, response_metadata={}),
    HumanMessage(content='How are you today?', additional_kwargs={}, response_metadata={}),
    AIMessage(content='Fine thanks!', additional_kwargs={}, response_metadata={})
]

In [29]:
from langchain_community.chat_message_histories import SQLChatMessageHistory

demo_ephemeral_chat_history_file = SQLChatMessageHistory(session_id='session002', connection_string='sqlite:///mem_sqlite.db')

demo_ephemeral_chat_history_file.add_user_message("Hey there! I'm Nemo.")
demo_ephemeral_chat_history_file.add_ai_message("Hello!")
demo_ephemeral_chat_history_file.add_user_message("How are you today?")
demo_ephemeral_chat_history_file.add_ai_message("Fine thanks!")

print(demo_ephemeral_chat_history_file.messages)

[
    HumanMessage(content="Hey there! I'm Nemo.", additional_kwargs={}, response_metadata={}),
    AIMessage(content='Hello!', additional_kwargs={}, response_metadata={}),
    HumanMessage(content='How are you today?', additional_kwargs={}, response_metadata={}),
    AIMessage(content='Fine thanks!', additional_kwargs={}, response_metadata={}),
    HumanMessage(content="Hey there! I'm Nemo.", additional_kwargs={}, response_metadata={}),
    AIMessage(content='Hello!', additional_kwargs={}, response_metadata={}),
    HumanMessage(content='How are you today?', additional_kwargs={}, response_metadata={}),
    AIMessage(content='Fine thanks!', additional_kwargs={}, response_metadata={})
]

In [8]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.runnables.history import RunnableWithMessageHistory

chat = ChatOpenAI(model="gpt-4o-mini")

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability. The provided chat history includes facts about the user you are speaking with.",
        ),
        ("placeholder", "{chat_history}"),
        ("user", "{input}"),
    ]
)

chain = prompt | print_input | chat

chain_with_message_history = RunnableWithMessageHistory(
    chain,
    lambda session_id: demo_ephemeral_chat_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [11]:
from langchain_core.runnables import RunnablePassthrough

def summarize_messages(chain_input):
    stored_messages = demo_ephemeral_chat_history.messages
    if len(stored_messages) == 0:
        return False
    summarization_prompt = ChatPromptTemplate.from_messages(
        [
            ("placeholder", "{chat_history}"),
            (
                "user",
                "Distill the above chat messages into a single summary message. Include as many specific details as you can.",
            ),
        ]
    )
    summarization_chain = summarization_prompt | print_input | chat

    summary_message = summarization_chain.invoke({"chat_history": stored_messages})

    demo_ephemeral_chat_history.clear()

    demo_ephemeral_chat_history.add_message(summary_message)

    return True


chain_with_summarization = (
    RunnablePassthrough.assign(messages_summarized=summarize_messages)
    | print_input | chain_with_message_history
)

In [14]:
chain_with_summarization.invoke(
    {"input": "What did I say my name was?"},
    {"configurable": {"session_id": "m001"}},
)

input:

ChatPromptValue(
    messages=[
        AIMessage(
            content='In the chat, Nemo greeted me and I asked how he was doing. He replied that he was fine and 
confirmed his name as Nemo.',
            additional_kwargs={'refusal': None},
            response_metadata={
                'token_usage': {
                    'completion_tokens': 28,
                    'prompt_tokens': 86,
                    'total_tokens': 114,
                    'completion_tokens_details': {'reasoning_tokens': 0},
                    'prompt_tokens_details': {'cached_tokens': 0}
                },
                'model_name': 'gpt-4o-mini-2024-07-18',
                'system_fingerprint': 'fp_f85bea6784',
                'finish_reason': 'stop',
                'logprobs': None
            },
            id='run-4e9a2bc9-92e8-45b6-9251-bf2492da0c3f-0',
            usage_metadata={'input_tokens': 86, 'output_tokens': 28, 'total_tokens': 114}
        ),
        HumanMessage(content='What did I say my name was?', additional_kwargs={}, response_metadata={}),
        AIMessage(
            content='You said your name was Nemo.',
            additional_kwargs={'refusal': None},
            response_metadata={
                'token_usage': {
                    'completion_tokens': 7,
                    'prompt_tokens': 81,
                    'total_tokens': 88,
                    'completion_tokens_details': {'reasoning_tokens': 0},
                    'prompt_tokens_details': {'cached_tokens': 0}
                },
                'model_name': 'gpt-4o-mini-2024-07-18',
                'system_fingerprint': 'fp_f85bea6784',
                'finish_reason': 'stop',
                'logprobs': None
            },
            id='run-fe278438-caf8-461d-9c0a-c17d3b267316-0',
            usage_metadata={'input_tokens': 81, 'output_tokens': 7, 'total_tokens': 88}
        ),
        HumanMessage(
            content='Distill the above chat messages into a single summary message. Include as many specific 
details as you can.',
            additional_kwargs={},
            response_metadata={}
        )
    ]
)

input:

{'input': 'What did I say my name was?', 'messages_summarized': True}

input:

ChatPromptValue(
    messages=[
        SystemMessage(
            content='You are a helpful assistant. Answer all questions to the best of your ability. The provided 
chat history includes facts about the user you are speaking with.',
            additional_kwargs={},
            response_metadata={}
        ),
        AIMessage(
            content='In the chat, Nemo greeted me and confirmed that he was doing fine when I asked how he was. He 
also stated that his name was Nemo.',
            additional_kwargs={'refusal': None},
            response_metadata={
                'token_usage': {
                    'completion_tokens': 30,
                    'prompt_tokens': 83,
                    'total_tokens': 113,
                    'completion_tokens_details': {'reasoning_tokens': 0},
                    'prompt_tokens_details': {'cached_tokens': 0}
                },
                'model_name': 'gpt-4o-mini-2024-07-18',
                'system_fingerprint': 'fp_f85bea6784',
                'finish_reason': 'stop',
                'logprobs': None
            },
            id='run-55449e8b-6256-4b47-8bc1-9cb7779112fc-0',
            usage_metadata={'input_tokens': 83, 'output_tokens': 30, 'total_tokens': 113}
        ),
        HumanMessage(content='What did I say my name was?', additional_kwargs={}, response_metadata={})
    ]
)

AIMessage(content='You said your name is Nemo.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 83, 'total_tokens': 90, 'completion_tokens_details': {'reasoning_tokens': 0}, 'prompt_tokens_details': {'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_f85bea6784', 'finish_reason': 'stop', 'logprobs': None}, id='run-4dd2df9a-83ff-4e65-8ecd-a1e4f86e9d5a-0', usage_metadata={'input_tokens': 83, 'output_tokens': 7, 'total_tokens': 90})

In [16]:
chain_with_summarization.invoke(
    {"input": "How old are you?"},
    {"configurable": {"session_id": "m001"}},
)

input:

ChatPromptValue(
    messages=[
        AIMessage(
            content='In the chat, you, named Nemo, greeted me and confirmed that you were doing fine when I 
inquired about your well-being.',
            additional_kwargs={'refusal': None},
            response_metadata={
                'token_usage': {
                    'completion_tokens': 27,
                    'prompt_tokens': 85,
                    'total_tokens': 112,
                    'completion_tokens_details': {'reasoning_tokens': 0},
                    'prompt_tokens_details': {'cached_tokens': 0}
                },
                'model_name': 'gpt-4o-mini-2024-07-18',
                'system_fingerprint': 'fp_f85bea6784',
                'finish_reason': 'stop',
                'logprobs': None
            },
            id='run-e046c25b-9602-4f45-8ed0-55a72c8285ee-0',
            usage_metadata={'input_tokens': 85, 'output_tokens': 27, 'total_tokens': 112}
        ),
        HumanMessage(content='How old are you?', additional_kwargs={}, response_metadata={}),
        AIMessage(
            content='I don’t have an age like a person does, but I was created by OpenAI and have been trained on 
data up until October 2023. How can I assist you today?',
            additional_kwargs={'refusal': None},
            response_metadata={
                'token_usage': {
                    'completion_tokens': 38,
                    'prompt_tokens': 77,
                    'total_tokens': 115,
                    'completion_tokens_details': {'reasoning_tokens': 0},
                    'prompt_tokens_details': {'cached_tokens': 0}
                },
                'model_name': 'gpt-4o-mini-2024-07-18',
                'system_fingerprint': 'fp_f85bea6784',
                'finish_reason': 'stop',
                'logprobs': None
            },
            id='run-26111326-e38f-43b9-a91d-3b502012b838-0',
            usage_metadata={'input_tokens': 77, 'output_tokens': 38, 'total_tokens': 115}
        ),
        HumanMessage(
            content='Distill the above chat messages into a single summary message. Include as many specific 
details as you can.',
            additional_kwargs={},
            response_metadata={}
        )
    ]
)

input:

{'input': 'How old are you?', 'messages_summarized': True}

input:

ChatPromptValue(
    messages=[
        SystemMessage(
            content='You are a helpful assistant. Answer all questions to the best of your ability. The provided 
chat history includes facts about the user you are speaking with.',
            additional_kwargs={},
            response_metadata={}
        ),
        AIMessage(
            content="In our chat, I, as an AI, identified myself as having been created by OpenAI and trained on 
data up to October 2023. You, named Nemo, greeted me and confirmed you were doing fine when I asked about your 
well-being. Additionally, when I was asked about my age, I explained that I don't have an age like a person.",
            additional_kwargs={'refusal': None},
            response_metadata={
                'token_usage': {
                    'completion_tokens': 73,
                    'prompt_tokens': 110,
                    'total_tokens': 183,
                    'completion_tokens_details': {'reasoning_tokens': 0},
                    'prompt_tokens_details': {'cached_tokens': 0}
                },
                'model_name': 'gpt-4o-mini-2024-07-18',
                'system_fingerprint': 'fp_f85bea6784',
                'finish_reason': 'stop',
                'logprobs': None
            },
            id='run-5ecbc3a8-a385-4bc8-9642-49dc3f9b2092-0',
            usage_metadata={'input_tokens': 110, 'output_tokens': 73, 'total_tokens': 183}
        ),
        HumanMessage(content='How old are you?', additional_kwargs={}, response_metadata={})
    ]
)

AIMessage(content="I don't have an age like a person does. I was created by OpenAI, and my training includes information up to October 2023. You could say I'm as old as the data and technology behind me!", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 43, 'prompt_tokens': 123, 'total_tokens': 166, 'completion_tokens_details': {'reasoning_tokens': 0}, 'prompt_tokens_details': {'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_f85bea6784', 'finish_reason': 'stop', 'logprobs': None}, id='run-cc616bcb-3579-45e9-9bc8-3bb3b41f8666-0', usage_metadata={'input_tokens': 123, 'output_tokens': 43, 'total_tokens': 166})

In [17]:
print(demo_ephemeral_chat_history.messages)

[
    AIMessage(
        content="In our chat, I, as an AI, identified myself as having been created by OpenAI and trained on data 
up to October 2023. You, named Nemo, greeted me and confirmed you were doing fine when I asked about your 
well-being. Additionally, when I was asked about my age, I explained that I don't have an age like a person.",
        additional_kwargs={'refusal': None},
        response_metadata={
            'token_usage': {
                'completion_tokens': 73,
                'prompt_tokens': 110,
                'total_tokens': 183,
                'completion_tokens_details': {'reasoning_tokens': 0},
                'prompt_tokens_details': {'cached_tokens': 0}
            },
            'model_name': 'gpt-4o-mini-2024-07-18',
            'system_fingerprint': 'fp_f85bea6784',
            'finish_reason': 'stop',
            'logprobs': None
        },
        id='run-5ecbc3a8-a385-4bc8-9642-49dc3f9b2092-0',
        usage_metadata={'input_tokens': 110, 'output_tokens': 73, 'total_tokens': 183}
    ),
    HumanMessage(content='How old are you?', additional_kwargs={}, response_metadata={}),
    AIMessage(
        content="I don't have an age like a person does. I was created by OpenAI, and my training includes 
information up to October 2023. You could say I'm as old as the data and technology behind me!",
        additional_kwargs={'refusal': None},
        response_metadata={
            'token_usage': {
                'completion_tokens': 43,
                'prompt_tokens': 123,
                'total_tokens': 166,
                'completion_tokens_details': {'reasoning_tokens': 0},
                'prompt_tokens_details': {'cached_tokens': 0}
            },
            'model_name': 'gpt-4o-mini-2024-07-18',
            'system_fingerprint': 'fp_f85bea6784',
            'finish_reason': 'stop',
            'logprobs': None
        },
        id='run-cc616bcb-3579-45e9-9bc8-3bb3b41f8666-0',
        usage_metadata={'input_tokens': 123, 'output_tokens': 43, 'total_tokens': 166}
    )
]